In [4]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
import torchmetrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from nltk.stem.snowball import SnowballStemmer

/home/pzalewsk/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


Load data

In [5]:
train = fetch_20newsgroups(subset='train', shuffle=True)
test = fetch_20newsgroups(subset='test', shuffle=True)
print(dir(train))
print(len(train.data))
print("Target names:", train.target_names)
print("------------------------------------")
print("Label:", train.target_names[train.target[0]])
print("Text:", train.data[0][:250])
print("------------------------------------")

['DESCR', 'data', 'filenames', 'target', 'target_names']
11314
Target names: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
------------------------------------
Label: rec.autos
Text: From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the ot
------------------------------------


Preprocessing

In [6]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
preprocess_pipeline = Pipeline([
    ("vect", StemmedCountVectorizer(stop_words="english", max_features=200)),
    ("tfidf", TfidfTransformer()),
])

In [7]:
train_data = preprocess_pipeline.fit_transform(train.data)

In [8]:
print(f"({len(train.data)}, {len(train.data[0])})")
print(train_data.shape)
train_labels = train.target
print(train_labels.shape)
print(train_labels[0])
number_of_labels = len(train.target_names)
print(number_of_labels)

(11314, 721)
(11314, 200)
(11314,)
7
20


Model definition

In [9]:
class DenseTextClassifier(pl.LightningModule):
    def __init__(self, network, loss_function, lr):
        super().__init__()
        self.network = network
        self.loss_function = loss_function
        self.lr = lr
        self.train_accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=number_of_labels)
        self.val_accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=number_of_labels)

    def forward(self, x):
        return self.network(x)

    def training_step(self, train_batch, _):
        inputs, targets = train_batch
        outputs = self(inputs)
        preds = torch.argmax(outputs, dim=1)
        
        train_loss = self.loss_function(outputs, targets)
        self.train_accuracy(preds, targets)
        self.log("train_loss", train_loss, prog_bar=True)
        self.log("train_acc", self.train_accuracy, prog_bar=True)
        return train_loss
    
    def validation_step(self, val_batch, _):
        inputs, targets = val_batch
        outputs = self(inputs)
        preds = torch.argmax(outputs, dim=1)
        
        val_loss = self.loss_function(outputs, targets)
        self.val_accuracy(preds, targets)
        self.log("val_loss", val_loss, prog_bar=True)
        self.log("val_acc", self.val_accuracy, prog_bar=True)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

In [10]:
class SimpleNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, output_size)
        )

    def forward(self, x):
        return self.network(x)

Learning

In [15]:
num_epochs = 10
input_size = train_data.shape[1]
output_size = number_of_labels
batch_size = 64
lr = 0.001
loss_function = nn.CrossEntropyLoss()
k_folds = 5


data_tensor = torch.from_numpy(train_data.toarray()).float()
labels_tensor = torch.from_numpy(train_labels).long()
dataset = TensorDataset(data_tensor, labels_tensor)

kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
val_accuracies = []
for fold, (train_ids, val_ids) in enumerate(kfold.split(data_tensor)):
    print(f'Fold: {fold}')

    train_loader = DataLoader(
                      dataset, 
                      batch_size=batch_size, 
                      sampler=SubsetRandomSampler(train_ids))
    val_loader = DataLoader(
                    dataset, 
                    batch_size=batch_size, 
                    sampler=SubsetRandomSampler(val_ids))

    network = SimpleNet(input_size, output_size)
    model = DenseTextClassifier(network=network, loss_function=loss_function, lr=lr)

    trainer = Trainer(max_epochs=num_epochs)
    trainer.fit(model, train_loader, val_loader)
    
    val_accuracies.append(trainer.callback_metrics.get("val_acc"))

average_val_accuracy = sum(val_accuracies) / len(val_accuracies)
print(f'Average Validation Accuracy across {k_folds} folds: {average_val_accuracy:.4f}')


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type               | Params
------------------------------------------------------
0 | network        | SimpleNet          | 239 K 
1 | loss_function  | CrossEntropyLoss   | 0     
2 | train_accuracy | MulticlassAccuracy | 0     
3 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
239 K     Trainable params
0         Non-trainable params
239 K     Total params
0.958     Total estimated model params size (MB)


Fold: 0


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/pzalewsk/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/home/pzalewsk/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


Average Validation Accuracy across 5 folds: 0.4570
